# madmom drum transcriptor

In [ ]:
!pip install numpy scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 92.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.3 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.3 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.3 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.3 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.3 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.3 which is incompatible.


In [ ]:
!pip install git+https://github.com/joshmiao1065/madmom.git

  Cloning https://github.com/joshmiao1065/madmom.git to /tmp/pip-req-build-ayd8wf06
  Running command git clone --filter=blob:none --quiet https://github.com/joshmiao1065/madmom.git /tmp/pip-req-build-ayd8wf06
  Resolved https://github.com/joshmiao1065/madmom.git to commit ffd66bb15d0e75b5ba0faa95ba85ee2964d54013
  Preparing metadata (setup.py) ... done


# drum transcriptor but running directly in colab

i used gemini to remove argparse because it was giving me a lot of issues and to instead use the input wav file as a direct input instead of having to go thru argparse

In [ ]:
#!/usr/bin/env python
# encoding: utf-8
"""
Drum transcription with recurrent (RNN), convolutional (CNN), or convolutional recurrent neural networks (CRNN).

"""
import numpy as np
from __future__ import absolute_import, division, print_function

from madmom.features import ActivationsProcessor
from madmom.features.drums import CRNNDrumProcessor, DrumPeakPickingProcessor
from madmom.features.notes import write_notes
from madmom.processors import IOProcessor


def main(audio_path, output_format=None, verbose=False, load=False, save=False, threshold=0.15, smooth=0, pre_avg=0.1, post_avg=0.01, pre_max=0.02, post_max=0.01, combine=0.02, delay=0, model=None, rand=False):
    """DrumTranscriptor"""

    # set immutable defaults
    fps = 100

    # set the suffix for midi files
    output_suffix = '.drums.txt'
    if output_format == 'midi':
        output_suffix = '.mid'

    # print arguments
    if verbose:
        print(f"Processing audio file: {audio_path}")
        print(f"Output format: {output_format}")
        print(f"Verbose: {verbose}")
        print(f"Load activations: {load}")
        print(f"Save activations: {save}")
        print(f"Threshold: {threshold}")
        print(f"Smooth: {smooth}")
        print(f"Pre-average: {pre_avg}")
        print(f"Post-average: {post_avg}")
        print(f"Pre-max: {pre_max}")
        print(f"Post-max: {post_max}")
        print(f"Combine: {combine}")
        print(f"Delay: {delay}")
        print(f"Model: {model}")
        print(f"Random: {rand}")


    # input processor
    if load:
        # load the activations from file
        in_processor = ActivationsProcessor(mode='r', verbose=verbose)
    else:
        # use a NN to predict the notes
        in_processor = CRNNDrumProcessor(model=model, rand=rand, verbose=verbose)

    # output processor
    if save:
        # save the NN note activations to file
        out_processor = ActivationsProcessor(mode='w', verbose=verbose)
    else:
        # perform peak picking on the activation function
        peak_picking = DrumPeakPickingProcessor(threshold=threshold, smooth=smooth, pre_avg=pre_avg, post_avg=post_avg, pre_max=pre_max,
                                           post_max=post_max, combine=combine, delay=delay, verbose=verbose)
        # output everything in the right format
        if output_format is None:
            output = write_notes
        else:
            raise ValueError('unknown output format: %s' % output_format)
        out_processor = [peak_picking, output]

    # create an IOProcessor
    processor = IOProcessor(in_processor, out_processor)

    # and call the processing function
    processor(audio_path)


if __name__ == '__main__':
    # This part will not be executed when calling main() from another cell
    print("Script is running directly.")
    # You would typically add code here to handle execution as a standalone script
    # For Colab, it's usually better to call main() from another cell

NameError: name 'float32' is not defined

In [ ]:
# Replace 'MDBDrums/MDB Drums/audio/full_mix/MusicDelta_Grunge_MIX.wav' with the path to your audio file
main("MDBDrums/MDB Drums/audio/full_mix/MusicDelta_Grunge_MIX.wav", model="CNN_3")

ValueError: fname must be a string or file handle

#Extract MDB dataset
## Split the data into training validation and test and attach to annotation files for evaluation

In [ ]:
!git clone https://github.com/CarlSouthall/MDBDrums.git

Cloning into 'MDBDrums'...
remote: Enumerating objects: 439, done.
remote: Total 439 (delta 0), reused 0 (delta 0), pack-reused 439 (from 1)
Receiving objects: 100% (439/439), 1.29 GiB | 26.18 MiB/s, done.
Resolving deltas: 100% (29/29), done.
Updating files: 100% (362/362), done.


In [ ]:
import os
import random

# Define the base directory for the dataset
base_dir = 'MDBDrums/MDB Drums'

# Define the directories containing audio and annotation files
audio_dirs = [
    os.path.join(base_dir, 'audio', 'drum_only'),
    os.path.join(base_dir, 'audio', 'full_mix')
]

# Update annotation directory to 'beats' and look for '.beats' files
annotation_dirs = [
    os.path.join(base_dir, 'annotations', 'beats')
]

# Get a list of all audio file paths
audio_files = []
for audio_dir in audio_dirs:
    for root, _, files in os.walk(audio_dir):
        for file in files:
            if file.endswith('.wav'):
                audio_files.append(os.path.join(root, file))

# Create a mapping between audio and annotation files
# Assuming annotation files have the same base name but different extensions and are in a specific annotation directory
file_pairs = []
for audio_file in audio_files:
    # Extract the base name from the audio file
    base_name = os.path.splitext(os.path.basename(audio_file))[0]

    # Construct potential annotation file paths in the 'beats' directory with '.beats' extension
    found_annotation = False
    for annotation_dir in annotation_dirs:
        annotation_file = os.path.join(annotation_dir, base_name + '.beats')

        if os.path.exists(annotation_file):
            file_pairs.append((audio_file, annotation_file))
            found_annotation = True
            break # Assume one annotation per audio file is sufficient

    if not found_annotation:
        # This warning might still appear if some audio files don't have corresponding .beats annotations
        # in the 'beats' directory, but it should be fewer than before.
        print(f"Warning: No matching annotation found for audio file: {audio_file}")


# Randomly shuffle the list of audio-annotation pairs
random.shuffle(file_pairs)

train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

total_files = len(file_pairs)
train_count = int(total_files * train_ratio)
val_count = int(total_files * val_ratio)
test_count = total_files - train_count - val_count # Ensure all files are included

# Split the shuffled list into training, validation, and testing sets
train_set = file_pairs[:train_count]
val_set = file_pairs[train_count:train_count + val_count]
test_set = file_pairs[train_count + val_count:]
_audio = [pair[0] for pair in file_pairs]
_annotation = [pair[1] for pair in file_pairs]

# Create separate lists for audio and annotation files for each set
train_audio = [pair[0] for pair in train_set]
train_annotations = [pair[1] for pair in train_set]

val_audio = [pair[0] for pair in val_set]
val_annotations = [pair[1] for pair in val_set]

test_audio = [pair[0] for pair in test_set]
test_annotations = [pair[1] for pair in test_set]

# Print the number of files in each set to verify the split
print(f"Total files found: {total_files}")
print(f"Training set size: {len(train_set)}")
print(f"Validation set size: {len(val_set)}")
print(f"Testing set size: {len(test_set)}")

# Display a few examples from each set (optional)
print("\nTraining set examples (audio, annotation):")
display(train_set[:5])

print("\nValidation set examples (audio, annotation):")
display(val_set[:5])

print("\nTesting set examples (audio, annotation):")
display(test_set[:5])

Total files found: 23
Training set size: 16
Validation set size: 3
Testing set size: 4

Training set examples (audio, annotation):


[('MDBDrums/MDB Drums/audio/full_mix/MusicDelta_Beatles_MIX.wav',
  'MDBDrums/MDB Drums/annotations/beats/MusicDelta_Beatles_MIX.beats'),
 ('MDBDrums/MDB Drums/audio/full_mix/MusicDelta_Hendrix_MIX.wav',
  'MDBDrums/MDB Drums/annotations/beats/MusicDelta_Hendrix_MIX.beats'),
 ('MDBDrums/MDB Drums/audio/full_mix/MusicDelta_CoolJazz_MIX.wav',
  'MDBDrums/MDB Drums/annotations/beats/MusicDelta_CoolJazz_MIX.beats'),
 ('MDBDrums/MDB Drums/audio/full_mix/MusicDelta_Shadows_MIX.wav',
  'MDBDrums/MDB Drums/annotations/beats/MusicDelta_Shadows_MIX.beats'),
 ('MDBDrums/MDB Drums/audio/full_mix/MusicDelta_FusionJazz_MIX.wav',
  'MDBDrums/MDB Drums/annotations/beats/MusicDelta_FusionJazz_MIX.beats')]


Validation set examples (audio, annotation):


[('MDBDrums/MDB Drums/audio/full_mix/MusicDelta_ModalJazz_MIX.wav',
  'MDBDrums/MDB Drums/annotations/beats/MusicDelta_ModalJazz_MIX.beats'),
 ('MDBDrums/MDB Drums/audio/full_mix/MusicDelta_FunkJazz_MIX.wav',
  'MDBDrums/MDB Drums/annotations/beats/MusicDelta_FunkJazz_MIX.beats'),
 ('MDBDrums/MDB Drums/audio/full_mix/MusicDelta_Country1_MIX.wav',
  'MDBDrums/MDB Drums/annotations/beats/MusicDelta_Country1_MIX.beats')]


Testing set examples (audio, annotation):


[('MDBDrums/MDB Drums/audio/full_mix/MusicDelta_Disco_MIX.wav',
  'MDBDrums/MDB Drums/annotations/beats/MusicDelta_Disco_MIX.beats'),
 ('MDBDrums/MDB Drums/audio/full_mix/MusicDelta_LatinJazz_MIX.wav',
  'MDBDrums/MDB Drums/annotations/beats/MusicDelta_LatinJazz_MIX.beats'),
 ('MDBDrums/MDB Drums/audio/full_mix/MusicDelta_Gospel_MIX.wav',
  'MDBDrums/MDB Drums/annotations/beats/MusicDelta_Gospel_MIX.beats'),
 ('MDBDrums/MDB Drums/audio/full_mix/MusicDelta_Britpop_MIX.wav',
  'MDBDrums/MDB Drums/annotations/beats/MusicDelta_Britpop_MIX.beats')]

## Verify the split and check that each set contains the relevant wav and annotation files

In [ ]:
import os

sets_to_check = {
    "Training": train_set,
    "Validation": val_set,
    "Testing": test_set
}

all_files_found = True

for set_name, data_set in sets_to_check.items():
    print(f"Checking {set_name} set...")
    for audio_file, annotation_file in data_set:
        if not os.path.exists(audio_file):
            print(f"Warning: Missing audio file in {set_name} set: {audio_file}")
            all_files_found = False
        if not os.path.exists(annotation_file):
            print(f"Warning: Missing annotation file in {set_name} set: {annotation_file}")
            all_files_found = False

if all_files_found:
    print("\nAll files in all sets were found.")
else:
    print("\nSome files were missing in one or more sets.")

## 23 training, 3 val, 4 test samples
# run on pre-trained model via DrumTranscriptor (CNN_3)

In [ ]:
import os

batch_files = ""
#replace test_audio with _audio to run on all collected audio files
for path in test_audio:
  batch_files += "\""+path+"\" "
print(batch_files)

#outputs stored in detections/ directory
drum_transcriptor_cmd = f"""DrumTranscriptor -m CNN_3 batch {batch_files} -o "detections" -s ".txt" """
print(drum_transcriptor_cmd)
os.system(drum_transcriptor_cmd)

"MDBDrums/MDB Drums/audio/full_mix/MusicDelta_Disco_MIX.wav" "MDBDrums/MDB Drums/audio/full_mix/MusicDelta_LatinJazz_MIX.wav" "MDBDrums/MDB Drums/audio/full_mix/MusicDelta_Gospel_MIX.wav" "MDBDrums/MDB Drums/audio/full_mix/MusicDelta_Britpop_MIX.wav" 
DrumTranscriptor -m CNN_3 batch "MDBDrums/MDB Drums/audio/full_mix/MusicDelta_Disco_MIX.wav" "MDBDrums/MDB Drums/audio/full_mix/MusicDelta_LatinJazz_MIX.wav" "MDBDrums/MDB Drums/audio/full_mix/MusicDelta_Gospel_MIX.wav" "MDBDrums/MDB Drums/audio/full_mix/MusicDelta_Britpop_MIX.wav"  -o "detections" -s ".txt" 
